In [1]:
import leafmap
from tqdm import tqdm
import datetime
import geopandas as gpd
import pandas as pd
import os

In [2]:
events_df = pd.read_csv('dateEventi.csv', sep=';')
events_df.head(2)

,Event,Aligned name,date,month,day,year,date2
0,Afghanistan Earthquake,afghanistan-earthquake22,2022-06-21,6,21,2022,"June 21, 2022"
1,Bay of Bengal Cyclone Mocha 2023,BayofBengal-Cyclone-Mocha-May-23,2023-05-23,5,23,2023,"May 23, 2023"


In [3]:
# Create a dictionary with the event name as key and the date as value

event2date = events_df.set_index('Aligned name')['date'].to_dict()
#dict_df

In [6]:
all_collections = leafmap.maxar_collections()
all_collections

['BayofBengal-Cyclone-Mocha-May-23',
 'Emilia-Romagna-Italy-flooding-may23',
 'Gambia-flooding-8-11-2022',
 'Hurricane-Fiona-9-19-2022',
 'Hurricane-Ian-9-26-2022',
 'Hurricane-Idalia-Florida-Aug23',
 'India-Floods-Oct-2023',
 'Indonesia-Earthquake22',
 'Kahramanmaras-turkey-earthquake-23',
 'Kalehe-DRC-Flooding-5-8-23',
 'Libya-Floods-Sept-2023',
 'Marshall-Fire-21-Update',
 'Maui-Hawaii-fires-Aug-23',
 'McDougallCreekWildfire-BC-Canada-Aug-23',
 'Morocco-Earthquake-Sept-2023',
 'NWT-Canada-Aug-23',
 'New-Zealand-Flooding22',
 'New-Zealand-Flooding23',
 'Sudan-flooding-8-22-2022',
 'afghanistan-earthquake22',
 'cyclone-emnati22',
 'ghana-explosion22',
 'kentucky-flooding-7-29-2022',
 'pakistan-flooding22',
 'shovi-georgia-landslide-8Aug23',
 'southafrica-flooding22',
 'tonga-volcano21',
 'volcano-indonesia21',
 'yellowstone-flooding22']

In [7]:
def get_pre_post_gdf(collection_id, event2date = event2date):
    #Create the geodataframe
    gdf = gpd.GeoDataFrame()
    for child_id in tqdm(leafmap.maxar_child_collections(collection_id)):
        current_gdf = leafmap.maxar_items(
            collection_id = collection_id,
            child_id = child_id,
            return_gdf=True,
            assets=['visual'],
        )
        gdf = pd.concat([gdf, current_gdf])

    #Retrieve the event date and split the geodataframe
    event_date = event2date[collection_id] #'2023-10-04'
    pre_gdf = gdf[gdf['datetime'] < event_date]
    post_gdf = gdf[gdf['datetime'] >= event_date]

    print('Collection_id:',collection_id,'\nEvent date:', event_date)

    if pre_gdf.shape[0] + post_gdf.shape[0] == gdf.shape[0]:
        print("OK: All items are accounted for\n")
    else:
        print("ERROR: Some items are missing!!!\n")

    print("pre_gdf", pre_gdf.shape)
    print("post_gdf", post_gdf.shape)

    return pre_gdf, post_gdf

def download_event(collection_id):
    pre_gdf, post_gdf = get_pre_post_gdf(collection_id)

    out_dir_root = "/mnt/data2/vaschetti_data/maxar/"
    leafmap.maxar_download(pre_gdf['visual'].to_list(), out_dir = os.path.join(out_dir_root, collection_id, 'pre', ""))
    leafmap.maxar_download(post_gdf['visual'].to_list(), out_dir = os.path.join(out_dir_root, collection_id, 'post', ""))

def reorganize(collection_id):
    pre_gdf, post_gdf = get_pre_post_gdf(collection_id)

    pre_childs = pre_gdf['catalog_id'].unique()
    post_childs = post_gdf['catalog_id'].unique()

    print('Pre-event images:', pre_childs, '\nPost-event images:', post_childs)

In [ ]:
download_event('Emilia-Romagna-Italy-flooding-may23')

In [11]:
out_dir_root = "/mnt/data2/vaschetti_data/maxar/"
for collection_id in all_collections[:10]:
    if os.path.exists(os.path.join(out_dir_root, collection_id)):
        print('Already downloaded: ', collection_id)
    else:
        try:
            download_event(collection_id)
        except:
            print('ERROR: ', collection_id)

Already downloaded:  BayofBengal-Cyclone-Mocha-May-23
Already downloaded:  Emilia-Romagna-Italy-flooding-may23
Already downloaded:  Gambia-flooding-8-11-2022
Already downloaded:  Hurricane-Fiona-9-19-2022


 42%|████▏     | 37/89 [34:43<48:48, 56.31s/it]  


ERROR:  Hurricane-Ian-9-26-2022


  0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
#To reorganize already existing events

#select the event
collection_id = 'Hurricane-Fiona-9-19-2022'

#Create the geodataframe
gdf = gpd.GeoDataFrame()
for child_id in tqdm(leafmap.maxar_child_collections(collection_id)):
    current_gdf = leafmap.maxar_items(
        collection_id = collection_id,
        child_id = child_id,
        return_gdf=True,
        assets=['visual'],
    )
    gdf = pd.concat([gdf, current_gdf])

event_date = event2date[collection_id]
pre_gdf = gdf[gdf['datetime'] < event_date]
post_gdf = gdf[gdf['datetime'] >= event_date]

print('Collection_id:',collection_id,'\nEvent date:', event_date)

if pre_gdf.shape[0] + post_gdf.shape[0] == gdf.shape[0]:
    print("OK: All items are accounted for\n")
else:
    print("ERROR: Some items are missing!!!\n")

print("pre_gdf", pre_gdf.shape)
print("post_gdf", post_gdf.shape)


pre_childs = pre_gdf['catalog_id'].unique()
post_childs = post_gdf['catalog_id'].unique()
print('Pre-event images:', pre_childs, '\nPost-event images:', post_childs)


In [63]:
# Create pre and post gdf for a single event

collection_id = 'Emilia-Romagna-Italy-flooding-may23'

#Create the geodataframe
gdf = gpd.GeoDataFrame()
for child_id in tqdm(leafmap.maxar_child_collections(collection_id)):
    current_gdf = leafmap.maxar_items(
        collection_id = collection_id,
        child_id = child_id,
        return_gdf=True,
        assets=['visual'],
    )
    gdf = pd.concat([gdf, current_gdf])

#Retrieve the event date and split the geodataframe
event_date = event2date[collection_id] #'2023-10-04'
pre_gdf = gdf[gdf['datetime'] < event_date]
post_gdf = gdf[gdf['datetime'] >= event_date]

print('Collection_id:',collection_id,'\nEvent date:', event_date)

if pre_gdf.shape[0] + post_gdf.shape[0] == gdf.shape[0]:
    print("OK: All items are accounted for\n")
else:
    print("ERROR: Some items are missing!!!\n")

print("pre_gdf", pre_gdf.shape)
print("post_gdf", post_gdf.shape)

100%|██████████| 11/11 [01:02<00:00,  5.64s/it]

Collection_id: Emilia-Romagna-Italy-flooding-may23 
Event date: 2023-05-23
OK: All items are accounted for

pre_gdf (125, 21)
post_gdf (120, 21)


In [10]:
leafmap.maxar_download(gdf['visual'].to_list()[:10], out_dir="/mnt/data2/vaschetti_data/maxar")

/mnt/data2/vaschetti_data/maxar/10300100E6747500/033111330333.tif already exists. Skipping...
/mnt/data2/vaschetti_data/maxar/10300100E6747500/033111331222.tif already exists. Skipping...
/mnt/data2/vaschetti_data/maxar/10300100E6747500/033111331223.tif already exists. Skipping...
/mnt/data2/vaschetti_data/maxar/10300100E6747500/033111331232.tif already exists. Skipping...
/mnt/data2/vaschetti_data/maxar/10300100E6747500/033111332111.tif already exists. Skipping...


Downloading...
From: https://maxar-opendata.s3.amazonaws.com/events/BayofBengal-Cyclone-Mocha-May-23/ard/46/033111332112/2023-05-22/10300100E6747500-visual.tif
To: /mnt/data2/vaschetti_data/maxar/10300100E6747500/033111332112.tif
100%|██████████| 4.65M/4.65M [00:01<00:00, 3.61MB/s]


Downloading...
From: https://maxar-opendata.s3.amazonaws.com/events/BayofBengal-Cyclone-Mocha-May-23/ard/46/033111332113/2023-05-22/10300100E6747500-visual.tif
To: /mnt/data2/vaschetti_data/maxar/10300100E6747500/033111332113.tif
100%|██████████| 59.0M/59.0M [00:05<00:00, 10.8MB/s]


Downloading...
From: https://maxar-opendata.s3.amazonaws.com/events/BayofBengal-Cyclone-Mocha-May-23/ard/46/033111332130/2023-05-22/10300100E6747500-visual.tif
To: /mnt/data2/vaschetti_data/maxar/10300100E6747500/033111332130.tif
100%|██████████| 4.14M/4.14M [00:01<00:00, 3.17MB/s]


Downloading...
From: https://maxar-opendata.s3.amazonaws.com/events/BayofBengal-Cyclone-Mocha-May-23/ard/46/033111332131/2023-05-22/10300100E6747500-visual.tif
To: /mnt/data2/vaschetti_data/maxar/10300100E6747500/033111332131.tif
100%|██████████| 53.2M/53.2M [00:04<00:00, 12.0MB/s]


Downloading...
From: https://maxar-opendata.s3.amazonaws.com/events/BayofBengal-Cyclone-Mocha-May-23/ard/46/033111332133/2023-05-22/10300100E6747500-visual.tif
To: /mnt/data2/vaschetti_data/maxar/10300100E6747500/033111332133.tif
100%|██████████| 29.5M/29.5M [00:03<00:00, 9.13MB/s]


In [24]:
leafmap.maxar_collection_url("Emilia-Romagna-Italy-flooding-may23")

'https://raw.githubusercontent.com/giswqs/maxar-open-data/master/datasets/Emilia-Romagna-Italy-flooding-may23.geojson'

In [20]:
gdf = leafmap.maxar_all_items("Emilia-Romagna-Italy-flooding-may23", return_gdf=True)

Processing (01 out of 11): 10300100BF164000 ...
Processing (02 out of 11): 10300100C48E0A00 ...


AttributeError: 'GeoDataFrame' object has no attribute 'append'

In [ ]:
leafmap.maxar_download(images[:5], out_dir="/mnt/data2/vaschetti_data/maxar")